# Titanic - Machine Learning from Disaster

Filename: titanic-model.ipynb \
Author: Timothy Holland \
Last updated: 17/05/2024 \
Kaggle competition: https://www.kaggle.com/competitions/titanic/data



#### 1. Data Preprocessing

In [16]:
import pandas as pd

# Loading dataset into DataFrames
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

# Specify numeric and non-numeric columns
numeric_cols = ['Pclass', 'Age', 'SibSp', 'Parch', 'Fare']
non_numeric_cols = ['Name', 'Sex', 'Ticket', 'Cabin', 'Embarked']

# Missing numerical values filled with average
train_df[numeric_cols] = train_df[numeric_cols].fillna(train_df[numeric_cols].mean())
test_df[numeric_cols] = test_df[numeric_cols].fillna(test_df[numeric_cols].mean())

# Missing non-numeric values filled with mode
for col in non_numeric_cols:
    train_df[col].fillna(train_df[col].mode()[0], inplace=True)
    test_df[col].fillna(test_df[col].mode()[0], inplace=True)

# Split training into features and target variable
x_train = train_df.drop(['PassengerId', 'Survived'], axis=1)
y_train = train_df['Survived']
# Split test into features and target variable
x_test = test_df.drop(['PassengerId'], axis=1)